In [3]:
from delta import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
builder = (SparkSession.builder
           .appName("data-skipping-delta-table")
           .master("spark://spark-master:7077")
           .config("spark.executor.memory", "512m")
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"))

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [5]:
%load_ext sparksql_magic
%config SparkSql.limit=20

In [6]:
# Create some sample data frames
# A large data frame with 1 million rows
df = (spark.range(0, 1000000)
            .withColumn("salary", 100*(rand() * 100).cast("int"))
            .withColumn("gender", when((rand() * 2).cast("int") == 0, "M").otherwise("F"))
            .withColumn("country_code", 
                        when((rand() * 4).cast("int") == 0, "US")
                        .when((rand() * 4).cast("int") == 1, "CN")
                        .when((rand() * 4).cast("int") == 2, "IN")
                        .when((rand() * 4).cast("int") == 3, "BR")
                        .otherwise('RU')))
df.show(5)

+---+------+------+------------+
| id|salary|gender|country_code|
+---+------+------+------------+
|  0|  4400|     M|          IN|
|  1|  3100|     M|          US|
|  2|  4700|     F|          US|
|  3|  2600|     M|          BR|
|  4|  8400|     F|          RU|
+---+------+------+------------+
only showing top 5 rows



In [7]:
(df.write
 .format("delta")
 .mode("overwrite")
 .save("../data/tmp/employee_salary"))

In [8]:
df = (spark.range(0, 1000)
            .withColumn("salary", 100*(rand() * 100).cast("int"))
            .withColumn("gender", when((rand() * 2).cast("int") == 0, "M").otherwise("F"))
            .withColumn("country_code", 
                        when((rand() * 4).cast("int") == 0, "US")
                        .when((rand() * 4).cast("int") == 1, "CN")
                        .when((rand() * 4).cast("int") == 2, "IN")
                        .when((rand() * 4).cast("int") == 3, "BR")
                        .otherwise('RU')))
(df.write
 .format("delta")
 .mode("append")
 .save("../data/tmp/employee_salary"))

In [14]:
%%sparksql
DESCRIBE HISTORY delta.`/opt/workspace/data/tmp/employee_salary`

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2023-09-30 15:03:18.364000,null,null,WRITE,"{'mode': 'Append', 'partitionBy': '[]'}",null,null,null,0,Serializable,True,"{'numOutputRows': '1000', 'numOutputBytes': '9597', 'numFiles': '2'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
0,2023-09-30 15:01:47.380000,null,null,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",null,null,null,null,Serializable,False,"{'numOutputRows': '1000000', 'numOutputBytes': '5402017', 'numFiles': '2'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0


In [15]:
%%sparksql
OPTIMIZE delta.`/opt/workspace/data/tmp/employee_salary`

path,metrics
file:/opt/workspace/data/tmp/employee_salary,"Row(numFilesAdded=1, numFilesRemoved=4, filesAdded=Row(min=5406862, max=5406862, avg=5406862.0, totalFiles=1, totalSize=5406862), filesRemoved=Row(min=4431, max=2701371, avg=1352903.5, totalFiles=4, totalSize=5411614), partitionsOptimized=1, zOrderStats=None, numBatches=1, totalConsideredFiles=4, totalFilesSkipped=0, preserveInsertionOrder=False, numFilesSkippedToReduceWriteAmplification=0, numBytesSkippedToReduceWriteAmplification=0, startTimeMs=1696086587316, endTimeMs=0, totalClusterParallelism=2, totalScheduledTasks=0, autoCompactParallelismStats=None, deletionVectorStats=None, numTableColumns=4, numTableColumnsWithStats=4)"


In [18]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

In [19]:
%%sparksql
VACUUM delta.`/opt/workspace/data/tmp/employee_salary` RETAIN 0 HOURS

Deleted 4 files and directories in a total of 1 directories.


path
file:/opt/workspace/data/tmp/employee_salary


In [22]:
spark.stop()